In [5]:
import yfinance as yf
import math
import pandas as pd
import numpy as np

def BASS(stock_symbol : str, benchmark_symbol : str, start : str, end : str):
    
    """
    BASS function for any stock and a benchmark.

    Parameters:
    1. stock_symbol : str
        - The stock symbol of the company.
    2. benchmark_symbol : str
    3. start : str
    4. end : str
    """
    # Prepare data
    df = pd.DataFrame()
    stock_data = yf.Ticker(stock_symbol).history(start=start, end=end)
    if stock_data.empty:
        print("No data found for stock symbol: ", stock_symbol)
        return None, None, None, None
    df['stock'] = stock_data['Close']
    df['benchmark'] = yf.Ticker(benchmark_symbol).history(start=start, end=end)['Close']
    
    df['benchmark_returns'] = df['benchmark'].pct_change(fill_method=None)
    df['stock_returns'] = df['stock'].pct_change(fill_method=None)

    df = df.dropna()

    # print(df)
    # Calculate beta
    cov = df['benchmark_returns'].cov(df['stock_returns'])
    var = df['benchmark_returns'].var()
    beta = cov / var
    beta = round(beta, 2)

    # Calculate beta (2)
    # returns = df[['stock_returns', 'benchmark_returns']]
    # matrix = returns.cov()
    # beta = matrix.iat[1,0] / matrix.iat[1,1]

    # Calculate alpha
    # Either annualized returns or absolute yearly returns.
    # If more than a year's data, use CAGR. ((Final Price / Initial Price) ^ (1 / Years)) - 1
    # Assume risk free rate = 0
    benchmark_yearly_returns = (df["benchmark_returns"].mean() * 252)
    stock_yearly_returns = (df["stock_returns"].mean() * 252)

    alpha = (stock_yearly_returns - (beta * benchmark_yearly_returns)) * 100
    alpha = round(alpha, 2)

    std_dev = (df['stock_returns'].std()) * 100
    std_dev = round(std_dev, 2)

    # Calculate Sharpe Ratio
    # SR = mu/Sigma
    avg_returns = df['stock_returns'].mean()
    std = df['stock_returns'].std()
    daily_SR = avg_returns / std

    annual_SR = daily_SR * math.sqrt(252)
    annual_SR = round(annual_SR, 2)

    return beta, alpha, std_dev, annual_SR

In [6]:
from datetime import datetime

all_stocks = ["C52.SI", "T39.SI", "S68.SI", "G13.SI", "V03.SI" , "U11.SI", "C07.SI" , "D05.SI", "Z74.SI", "D01.SI", "O39.SI", "S63.SI", "A17U.SI" , "BN4.SI","BS6.SI", "M44U.SI", "H78.SI", 
"Y92.SI", "C38U.SI", "U14.SI", "N2IU.SI" , "F34.SI" , "C09.SI" ,
"J36.SI", "S58.SI" , "C6L.SI", "U96.SI" ,
"1810.HK", "9999.HK", "7500.HK", "9618.HK", "1024.HK", "3690.HK", "6618.HK"]

benchmark = "^STI"
# start = "2010-01-01"
# end = datetime.now().strftime("%Y-%m-%d")
# start = "2023-01-01"
# end = "2024-01-01"

# start="2023-08-30"
# end="2024-08-31"

end = (datetime.now() - pd.DateOffset(days=1)).strftime("%Y-%m-%d")
start = (datetime.now() - pd.DateOffset(years=1, days = 1)).strftime("%Y-%m-%d")

results = []

for s in all_stocks:
    beta, alpha, std_dev, annual_SR = BASS(s, benchmark, start, end)
    # print(f"Stock: {s}, Beta: {beta}, Alpha: {alpha}, Std Dev: {std_dev}, Annual SR: {annual_SR}")
    if beta is not None:
        results.append({
            "stock" : s,
            "beta" : beta,
            "alpha(%)" : alpha,
            "standard_dev(%)" : std_dev,
            "sharpe" : annual_SR
        })

output_df = pd.DataFrame(results)
output_df.sort_values(by='sharpe', ascending=False)

$T39.SI: possibly delisted; no timezone found


No data found for stock symbol:  T39.SI


,stock,beta,alpha(%),standard_dev(%),sharpe
9,O39.SI,1.02,16.08,0.84,1.90
7,Z74.SI,0.92,26.03,1.18,1.83
6,D05.SI,1.16,17.12,0.98,1.76
23,S58.SI,0.83,31.40,1.53,1.60
1,S68.SI,0.66,12.19,0.82,1.39
26,1810.HK,0.83,47.93,2.60,1.37
4,U11.SI,0.99,9.45,0.85,1.36
13,BS6.SI,0.69,41.19,2.28,1.31
0,C52.SI,0.47,15.54,1.00,1.24
10,S63.SI,0.90,11.29,1.00,1.22


In [7]:
yf.Ticker(ticker = "T39.SI").history(start=start, end=end).empty

$T39.SI: possibly delisted; no timezone found


True